In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

## Aggregate the regular season data for each team

In [ ]:
reg_season = pd.read_csv('../input/mens-march-mania-2022/MDataFiles_Stage1/MRegularSeasonDetailedResults.csv')
reg_season.head()

In [ ]:
w_cols = ['Season', 'WTeamID', 'WLoc', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR', 'WAst', 'WTO', 'WStl', 'WBlk', 'WPF']
l_cols = ['Season', 'LTeamID', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3', 'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF']
cols = ['Season', 'TeamID', 'FGM', 'FGA', 'FGM3', 'FGA3', 'FTM', 'FTA', 'OR', 'DR', 'Ast', 'TO', 'Stl', 'Blk', 'PF']

w_stats = reg_season[w_cols].groupby(['Season', 'WTeamID']).mean().reset_index()
w_stats.columns = cols

l_stats = reg_season[l_cols].groupby(['Season', 'LTeamID']).mean().reset_index()
l_stats.columns = cols

reg_season_stats = pd.concat([w_stats, l_stats]).groupby(['Season', 'TeamID']).mean().reset_index()
reg_season_stats['id'] = reg_season_stats.Season.astype(str) + reg_season_stats.TeamID.astype(str)
reg_season_stats.head()

## Aggregate the massey rank for each team

In [ ]:
ranks = pd.read_csv('../input/mens-march-mania-2022/MDataFiles_Stage1/MMasseyOrdinals.csv')

ranks_agg = ranks.groupby(['Season', 'TeamID']).agg({'OrdinalRank': ['mean', 'min', 'max']})
ranks_agg.columns = ['_'.join(col) for col in ranks_agg.columns]

team_ranks = ranks_agg.reset_index()
team_ranks['id'] = team_ranks.Season.astype(str) + team_ranks.TeamID.astype(str)
team_ranks.head()

In [ ]:
pd.read_csv('../input/mens-march-mania-2022/MDataFiles_Stage1/MNCAATourneyCompactResults.csv').shape

## Combine the features

In [ ]:
def is_winning(wteam, lteam):
    if wteam < lteam:
        return 1
    else:
        return 0
    
    
march_madness = pd.read_csv('../input/mens-march-mania-2022/MDataFiles_Stage1/MNCAATourneyCompactResults.csv')
train = march_madness
train['is_win'] = train.apply(lambda x: is_winning(x['WTeamID'], x['LTeamID']), axis=1)
train['team_a'] = train.Season.astype(str) + train.WTeamID.astype(str)
train['team_b'] = train.Season.astype(str) + train.LTeamID.astype(str)
train = train.drop(['WScore', 'LScore'], axis=1)
train = pd.merge(train, team_ranks, left_on='team_a', right_on='id').merge(team_ranks, left_on='team_b', right_on='id', suffixes=('_teama', '_teamb'))
train = train.drop(['Season_x', 'Season_y', 'id_teama', 'id_teamb', 'TeamID_teama', 'TeamID_teamb'], axis=1)
train = pd.merge(train, reg_season_stats, left_on='team_a', right_on='id').merge(reg_season_stats, left_on='team_b', right_on='id', suffixes=('_teama', '_teamb'))
train = train.drop(['Season_x', 'Season_y', 'team_a', 'team_b', 'TeamID_teama', 'TeamID_teamb', 'id_teama', 'id_teamb'], axis=1)
train = train.drop(['DayNum', 'WTeamID', 'LTeamID', 'NumOT', 'Season', 'WLoc'], axis=1)
print(train.shape)
train.head()

## Model Implementation

In [ ]:
from sklearn.model_selection import train_test_split

X = train.drop(['is_win'], axis=1)
y = train.is_win

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print(classification_report(y_test, y_pred))

## Predict The Result

In [ ]:
def return_value(data, data_type=None):
    data = data.split('_')
    if data_type == 'Team A':
        return str(data[0]) + str(data[1])
    else:
        return str(data[0]) + str(data[2])


test = pd.read_csv("../input/mens-march-mania-2022/MDataFiles_Stage1/MSampleSubmissionStage1.csv")

test['team_a'] = test.apply(lambda x: return_value(x.ID, 'Team A'), axis=1)
test['team_b'] = test.apply(lambda x: return_value(x.ID), axis=1)
test = pd.merge(test, team_ranks, left_on='team_a', right_on='id').merge(team_ranks, left_on='team_b', right_on='id', suffixes=('_teama', '_teamb'))
test = pd.merge(test, reg_season_stats, left_on='team_a', right_on='id').merge(reg_season_stats, left_on='team_b', right_on='id', suffixes=('_teama', '_teamb'))
test = test.drop(['Season_teama', 'Season_teamb', 'team_a', 'team_b', 'TeamID_teama', 'TeamID_teamb', 'id_teama', 'id_teamb'], axis=1)

test.head()

In [ ]:
X = test.drop(['ID', 'Pred'], axis=1)

test['Pred'] = model.predict_proba(X)[:, 1]
test = test[['ID', 'Pred']]
test.head()

In [ ]:
test.to_csv('submission.csv', index=False)

In [ ]:
pd.read_csv('submission.csv')